<a href="https://colab.research.google.com/github/irfankhan745/Daily-data-analytics-practice/blob/main/Pandas_prectice_day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [57]:
ipl = pd.read_csv('/content/deliveries_updated_mens_ipl.csv')
pd.set_option('display.max_column',None)
ipl.head(2)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,date
0,335982,1,0.1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2008-04-18
1,335982,1,0.2,0,2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18


# Task
Formulate several challenging questions about the `ipl` DataFrame that require advanced pandas operations to answer, without providing the solutions. Then, summarize these questions.

## Present Challenging Questions

### Subtask:
Formulate and present several challenging questions related to the `ipl` DataFrame that require advanced pandas operations to answer, without providing the solutions.


Cumulative Runs and Highest Individual Scores: Formulate a question that requires calculating the cumulative runs scored by each batsman in each match, identifying their highest individual score within that match, and then finding the batsman with the most instances of scoring above a certain threshold (e.g., 50 runs) across all seasons. This should involve grouping, cumulative calculations, and filtering.

In [ ]:
ipl.columns

Index(['matchId', 'inning', 'over_ball', 'over', 'ball', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty',
       'dismissal_kind', 'player_dismissed', 'date'],
      dtype='object')

In [ ]:
runs_per_match = (
    ipl
    .groupby(['batsman', 'matchId'], as_index=False)['batsman_runs']
    .sum()
    .rename(columns={'batsman_runs': 'match_runs'})
)
runs_per_match.sort_values(by='match_runs',ascending=False).head(2)

fifty_plus = runs_per_match[runs_per_match['match_runs'] >= 50]

fifty_plus_count = fifty_plus.groupby('batsman')['batsman'].count()
fifty_plus_count

,batsman
batsman,
A Badoni,2
A Symonds,6
AA Jhunjhunwala,1
AB de Villiers,44
AC Blizzard,1
...,...
WP Saha,14
Y Venugopal Rao,3
YBK Jaiswal,9


In [26]:
legel_out.columns

Index(['matchId', 'inning', 'over_ball', 'over', 'ball', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty',
       'dismissal_kind', 'player_dismissed', 'date'],
      dtype='object')

Bowler Performance in Unique Matches: Create a question that involves analyzing bowler performance. Specifically, identify bowlers who have taken wickets (dismissal_kind is not NaN and dismissal_kind not in 'run out', 'retired hurt') in at least 50 unique matches, and for these bowlers, calculate their average economy rate (runs conceded per over) in matches where they bowled at least 3 overs. This will require conditional grouping, counting unique values, and calculating a custom metric.

In [106]:
legal_balls = ipl
legel_balls = legal_balls[legal_balls['isWide'] == 0]




bawler_match = legal_balls.groupby(['bowler','matchId']).agg(ball=('ball','count'),runs=('batsman_runs','sum')).reset_index()

bawler_match['match_count'] = bawler_match['matchId'].nunique()

bawler_match = bawler_match[bawler_match['match_count'] >= 50]
bawler_match['over'] = round(bawler_match['ball'] / 6)
bawler_match = bawler_match[bawler_match['over'] >= 3]

bawler_match['avg'] = round(bawler_match['runs'] / bawler_match['over'])
bawler_match.sort_values(by='avg').head(2)


,bowler,matchId,ball,runs,match_count,over,avg
7243,P Kumar,392182,21,4,1024,4.0,1.0
1166,Akash Madhwal,1370351,21,5,1024,4.0,1.0


Death Over Specialist Batsmen: Develop a question to determine the 'death over specialist' batsmen. This should involve identifying batsmen who have scored a high percentage of their total runs in overs 16-20 across all matches they've played. The question should also require identifying the top 10 such batsmen based on their total runs scored in these death overs. This will necessitate filtering by over number, calculating percentages, and sorting.

In [148]:
#Batsman crear runs
total_runs = (ipl.groupby('batsman', as_index=False).agg(total_runs=('batsman_runs', 'sum')))

#create copy
death_over = ipl

#filtring
death_over = death_over[death_over['over'] >= 16]

#Death over = count balls and sum runs by player
death_over = death_over.groupby(['batsman'],as_index=False).agg(balls = ('ball','count'),runs =('batsman_runs','sum')).reset_index()
#death over runs per(%)
death_over['avg'] = (death_over['runs'] / death_over['balls']) * 100

#mearged
mearged = death_over.merge(total_runs, on='batsman')

#create runs and death over runs per(%)
mearged['per'] = (mearged['runs'] / mearged['total_runs'] ) * 100

# Top 10 death over runs maker
mearged.sort_values(by='per',ascending=False).head(10)




,index,batsman,balls,runs,avg,total_runs,per
553,553,Yudhvir Singh,2,1,50.000000,1,100.0
528,528,VG Arora,16,8,50.000000,8,100.0
523,523,Umran Malik,16,23,143.750000,23,100.0
517,517,TU Deshpande,13,21,161.538462,21,100.0
502,502,T Shamsi,4,2,50.000000,2,100.0
505,505,T Thushara,19,12,63.157895,12,100.0
501,501,T Natarajan,5,3,60.000000,3,100.0
491,491,Shivam Sharma,3,5,166.666667,5,100.0
448,448,SB Jakati,30,28,93.333333,28,100.0
461,461,SM Harwood,14,9,64.285714,9,100.0
